<a href="https://colab.research.google.com/github/navnee7h/AI-generated-Text-Detection/blob/main/AI_Detector_Random_Forest_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🔘AI Generated text detection ML model**

## **🔗Linking  google drive to google colab**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd

/content


# **Load data**

In [ ]:
import pandas as pd
df = pd.read_csv("drive/My Drive/GPT-wiki-intro.csv")
df

,id,url,title,wiki_intro,generated_intro,title_len,wiki_intro_len,generated_intro_len,prompt,generated_text,prompt_tokens,generated_text_tokens
0,63064638,https://en.wikipedia.org/wiki/Sexhow%20railway...,Sexhow railway station,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,3,174,78,200 word wikipedia style introduction on 'Sexh...,"located in the town of Sexhow, on the Cumbria...",25,88
1,279621,https://en.wikipedia.org/wiki/Eti%C3%A4inen,Etiäinen,"In Finnish folklore, all places and things, an...","In Finnish folklore, all places and things, an...",1,187,80,200 word wikipedia style introduction on 'Etiä...,"animate or inanimate, have a spirit or ""etiäi...",26,101
2,287229,https://en.wikipedia.org/wiki/Inverse%20functi...,Inverse function theorem,"In mathematics, specifically differential calc...","In mathematics, specifically differential calc...",3,170,59,200 word wikipedia style introduction on 'Inve...,function theorem states that for every real-v...,26,65
3,26712375,https://en.wikipedia.org/wiki/Stepping%20on%20...,Stepping on Roses,is a Japanese shōjo manga series written and i...,is a Japanese shōjo manga series written and i...,3,335,121,200 word wikipedia style introduction on 'Step...,and illustrated by Maki Fujii. The series fol...,26,150
4,38894426,https://en.wikipedia.org/wiki/Rob%20Bradley,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...","Robert Milner ""Rob"" Bradley, Jr. (born August ...",2,170,136,200 word wikipedia style introduction on 'Rob ...,"29, 1973) is an American former professional ...",28,162
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,44173767,https://en.wikipedia.org/wiki/Randy%20Borum,Randy Borum,Randy Borum is a Professor and Coordinator of ...,Randy Borum is a Professor and Coordinator of ...,2,185,71,200 word wikipedia style introduction on 'Rand...,of the Master of Fine Arts Program in Creativ...,25,92
149996,33564134,https://en.wikipedia.org/wiki/Sa%27och%20language,Sa'och language,"Sa'och (, also, ""Sauch"") is an endangered, nea...","Sa'och (, also, ""Sauch"") is an endangered, nuc...",2,175,134,200 word wikipedia style introduction on 'Sa'o...,"nuclear-speaking, isolate language of the Ath...",33,184
149997,4219548,https://en.wikipedia.org/wiki/Philip%20Hanawalt,Philip Hanawalt,Philip C. Hanawalt (born 1931) is an American ...,Philip C. Hanawalt (born 1931) is an American ...,2,166,191,200 word wikipedia style introduction on 'Phil...,American graphic artist and illustrator. He i...,30,272
149998,2625970,https://en.wikipedia.org/wiki/Vossius%20Gymnasium,Vossius Gymnasium,Vossius Gymnasium is a public gymnasium in Ams...,Vossius Gymnasium is a public gymnasium in the...,2,168,108,200 word wikipedia style introduction on 'Voss...,"the town of Vossius, Netherlands. It is named...",32,147


# **Data Preprocessing**


In [ ]:
df.drop(['url','title_len','wiki_intro_len','generated_intro_len','prompt','generated_text','prompt_tokens','generated_text_tokens'],axis=1,inplace=True)
print(df)

              id                     title  \
0       63064638    Sexhow railway station   
1         279621                  Etiäinen   
2         287229  Inverse function theorem   
3       26712375         Stepping on Roses   
4       38894426               Rob Bradley   
...          ...                       ...   
149995  44173767               Randy Borum   
149996  33564134           Sa'och language   
149997   4219548           Philip Hanawalt   
149998   2625970         Vossius Gymnasium   
149999  32345102           Simone Stratigo   

                                               wiki_intro  \
0       Sexhow railway station was a railway station b...   
1       In Finnish folklore, all places and things, an...   
2       In mathematics, specifically differential calc...   
3       is a Japanese shōjo manga series written and i...   
4       Robert Milner "Rob" Bradley, Jr. (born August ...   
...                                                   ...   
149995  Randy Borum 

## **Splitting the single dataframe into human generated and ai generated text,  adding the 'result' column**

In [ ]:
dfh=df.copy()
dfh.drop(['generated_intro'],axis=1,inplace=True)
dfh=dfh.head(50000)
dfh.rename(columns = {'wiki_intro':'text'}, inplace = True)
result=[]
for i in range(50000):
  result.append("Human Written")
dfh['result']=result
print(dfh)

             id                     title  \
0      63064638    Sexhow railway station   
1        279621                  Etiäinen   
2        287229  Inverse function theorem   
3      26712375         Stepping on Roses   
4      38894426               Rob Bradley   
...         ...                       ...   
49995   1119629            Word of wisdom   
49996   4096346                 Ski Beatz   
49997   1526322            Crispian Mills   
49998   4592607               Russell Ash   
49999  42161771  Afterglow Maverick Sabre   

                                                    text         result  
0      Sexhow railway station was a railway station b...  Human Written  
1      In Finnish folklore, all places and things, an...  Human Written  
2      In mathematics, specifically differential calc...  Human Written  
3      is a Japanese shōjo manga series written and i...  Human Written  
4      Robert Milner "Rob" Bradley, Jr. (born August ...  Human Written  
...            

In [ ]:
dfg=df.copy()
dfg.drop(['wiki_intro'],axis=1,inplace=True)
dfg=dfg.head(50000)
dfg.rename(columns = {'generated_intro':'text'}, inplace = True)
result=[]
for i in range(50000):
  result.append("AI Written")
dfg['result']=result
print(dfg)

             id                     title  \
0      63064638    Sexhow railway station   
1        279621                  Etiäinen   
2        287229  Inverse function theorem   
3      26712375         Stepping on Roses   
4      38894426               Rob Bradley   
...         ...                       ...   
49995   1119629            Word of wisdom   
49996   4096346                 Ski Beatz   
49997   1526322            Crispian Mills   
49998   4592607               Russell Ash   
49999  42161771  Afterglow Maverick Sabre   

                                                    text      result  
0      Sexhow railway station was a railway station l...  AI Written  
1      In Finnish folklore, all places and things, an...  AI Written  
2      In mathematics, specifically differential calc...  AI Written  
3      is a Japanese shōjo manga series written and i...  AI Written  
4      Robert Milner "Rob" Bradley, Jr. (born August ...  AI Written  
...                              

<ipython-input-85-0ca0301a1217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfg.rename(columns = {'generated_intro':'text'}, inplace = True)
<ipython-input-85-0ca0301a1217>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfg['result']=result


## **Combining the two dataframes into a single dataset**

In [ ]:
ds=pd.concat([dfh,dfg])
ds.shape

(100000, 4)

In [ ]:
print(ds)

             id                     title  \
0      63064638    Sexhow railway station   
1        279621                  Etiäinen   
2        287229  Inverse function theorem   
3      26712375         Stepping on Roses   
4      38894426               Rob Bradley   
...         ...                       ...   
49995   1119629            Word of wisdom   
49996   4096346                 Ski Beatz   
49997   1526322            Crispian Mills   
49998   4592607               Russell Ash   
49999  42161771  Afterglow Maverick Sabre   

                                                    text         result  
0      Sexhow railway station was a railway station b...  Human Written  
1      In Finnish folklore, all places and things, an...  Human Written  
2      In mathematics, specifically differential calc...  Human Written  
3      is a Japanese shōjo manga series written and i...  Human Written  
4      Robert Milner "Rob" Bradley, Jr. (born August ...  Human Written  
...            

# **Data Preparation**

## **Data Seperation as x and y**

In [ ]:
y=ds['result']
print(y)

0        Human Written
1        Human Written
2        Human Written
3        Human Written
4        Human Written
             ...      
49995       AI Written
49996       AI Written
49997       AI Written
49998       AI Written
49999       AI Written
Name: result, Length: 100000, dtype: object


In [ ]:
x=ds.drop(['result'], axis=1)
print(x)

             id                     title  \
0      63064638    Sexhow railway station   
1        279621                  Etiäinen   
2        287229  Inverse function theorem   
3      26712375         Stepping on Roses   
4      38894426               Rob Bradley   
...         ...                       ...   
49995   1119629            Word of wisdom   
49996   4096346                 Ski Beatz   
49997   1526322            Crispian Mills   
49998   4592607               Russell Ash   
49999  42161771  Afterglow Maverick Sabre   

                                                    text  
0      Sexhow railway station was a railway station b...  
1      In Finnish folklore, all places and things, an...  
2      In mathematics, specifically differential calc...  
3      is a Japanese shōjo manga series written and i...  
4      Robert Milner "Rob" Bradley, Jr. (born August ...  
...                                                  ...  
49995  In Christianity, the word of wisdom is 

## **Data Splitting**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ds['text'], ds['result'],test_size=0.2, random_state= 42)

In [ ]:
x_train

25220    Sara Niemietz  (born June 7, 1992) is an Ameri...
48955    Juneau Mountain Rescue (JMR) is a mountain sea...
44966    Erich Wolf Segal (June 16, 1937January 17, 201...
13568    Green jobs or green-collared jobs  (sustainabi...
42727    Nicolas Jarry (1620 – 1670) was a French calli...
                               ...                        
6265     Mount Moffit is a peak in the Alaska Range in ...
4886     The Łódź Jewish Cemetery, also known as the Łó...
26820    Herbert Blaché (5 October 1882 – 23 October 19...
860      The Flat Tops is a mountain range located in C...
15795    Mechanization is the process of changing from ...
Name: text, Length: 80000, dtype: object

In [ ]:
x_test

25721    The tapping rate is a psychological test used ...
30184    Explorer 55, also called as AE-E (Atmospheric ...
19864    John Andrews Murrell (1806 – November 21, 1844...
26699    William Burke Belknap the younger (1885–1965) ...
42991    A shallow donor refers to a donor who donates ...
                               ...                        
32595    Electric Angels was an American, Los Angeles-b...
29313    The Cordeaux Dam is a heritage-listed dam in C...
37862    The Nemean lion (;  Neméos léōn; ) was a vicio...
3421     Jean-Marc Coicaud is a French and American scu...
42410    Roland Emmerich (; born 10 November 1955) is a...
Name: text, Length: 20000, dtype: object

In [ ]:
y_train

25220       AI Written
48955    Human Written
44966    Human Written
13568    Human Written
42727       AI Written
             ...      
6265     Human Written
4886        AI Written
26820       AI Written
860      Human Written
15795    Human Written
Name: result, Length: 80000, dtype: object

In [ ]:
y_test

25721       AI Written
30184       AI Written
19864    Human Written
26699       AI Written
42991       AI Written
             ...      
32595    Human Written
29313    Human Written
37862    Human Written
3421        AI Written
42410    Human Written
Name: result, Length: 20000, dtype: object

# **Model Building**

## **Random Forest Model**

### **Importing necessary libraries**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

### **Extracting features using TF-IDF**

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

### **Model Creation**

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

### **Model training**

In [ ]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# **Model Evaluation**

In [ ]:
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="macro")
recall = recall_score(y_test, y_pred, average="macro")

In [ ]:

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.89835
Precision: 0.8987360870492084
Recall: 0.8984056304689733


# **Saving trained model to disk**

In [104]:
import pickle
# save the model as a pickle file
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)


In [105]:
from google.colab import files

files.download('rf_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Testing the model with a random text**

In [100]:
# random_text = '''
# This is the final step of dataset generation. At this step, we augment the anchor dataset with human written or GPT generated data. The most important part in this step is coming up with the prompt used for generating the response from GPT or question that will be answered by humans. For finalizing the prompt we can leverage OpenAI Playground to test out different prompts with different models, temperature, frequency penalty and presence penalty. To increase the diversity of the dataset, we can finalize n prompts and use them uniformly to get the responses. In case of human responses, we would want to finalize the question by giving different variations of the questions to small survey population and then checking the results to finalize the n questions. Once the prompts or the questions are finalized, we can use the OpenAI API to generate the GPT generated responses or use service such as Mechanical Turk to get the human written responses.

# '''

### **Vectorising the text with the same TfidfVectorizer used to fit the training data**

In [101]:
# random_text_tfidf = tfidf.transform([random_text])

### **Using the trained Random Forest model to make predictions**

In [102]:
# prediction = rf_model.predict(random_text_tfidf)
# print("Prediction:", prediction)